# Google Maps

In [1]:
#Importamos las librerías necesarias 
import pandas as pd
import json
import matplotlib.pyplot as plt
import re
import glob

## Metadata

Vamos a realizar el proceso para el archivo 1.json, y luego crear una función para hacer con todos los json.

In [2]:
#Leemos el archivo de matadata numero 1
df1=pd.read_json('Data/Metadata/1.json', lines=True)

In [3]:
#Observamos como se cargaron los datos
df1.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [4]:
#Analizamos las dimensiones del dataframe
df1.shape

(275001, 15)

In [5]:
#Selecciono las columnas que voy a utilizar
df1 = df1[['name', 'address', 'gmap_id', 'category', 'avg_rating']]

In [ ]:
#Analizo que se hayan seleccionado correctamente las columnas
df1.head()

In [7]:
#Reviso si hay valores nulos
df1.isnull().sum()

name              7
address       10062
gmap_id           0
category       2261
avg_rating        0
dtype: int64

Eliminamos los nulos en address y en category, porque no me permitirán filtrar la información ni por el estado de Florida ni por categoría restaurante.

In [8]:
#Elimino los nulos en address
df1 = df1.dropna(subset=['address'])

In [9]:
#Elimino los nulos en category
df1 = df1.dropna(subset=['category'])

In [10]:
#Verifico si aún tengo nulos
df1.isnull().sum()

name          0
address       0
gmap_id       0
category      0
avg_rating    0
dtype: int64

Voy a seleccionar la a partir de address la información sobre el Estado. 

In [11]:
# La columna state la reemplazo por address y normalizo por el ISO del estado
df1['state'] = df1['address'].str.upper()
df1['state'] = df1['state'].str.replace('FLORIDA', 'FL')

In [12]:
# Revisar los registros que contienen el ISO o el nombre de los estados y reemplazar por el ISO
df1['state'] = df1.apply(lambda row: 'FL' if row['state'] is not None and ('FL' in row['state'] or 'FLORIDA' in row['state']) else row['state'], axis=1)

# Filtro por etado de Florida
df1 = df1[df1['state'].isin([ 'FL'])]

In [13]:
# Lista de palabras para el filtro de category
palabras = ['restaurant', 'restaurants']

In [14]:
# Función para buscar coincidencias de palabras clave en la lista de strings
def buscar_coincidencias(lista):
    if lista is None:
        return False
    else:
        texto = ' '.join(lista).lower()
        for palabra in palabras:
            if re.search(r'\b' + re.escape(palabra.lower()) + r'\b', texto):
                return True
        return False

In [15]:
# Verificar si alguna palabra clave está presente en las listas de strings de la columna 'category'
df1['contains_keyword'] = df1['category'].apply(buscar_coincidencias)      

In [16]:
# Filtrar el DataFrame para obtener solo las filas que contienen palabras clave (los que sean TRUE)
df1 = df1[df1['contains_keyword']]

In [17]:
# Borrar duplicados de gmap_id:
df1.drop_duplicates(subset="gmap_id", inplace=True)

In [18]:
df1.shape

(839, 7)

A continuación presentamos la función para automatizar el proceso para todos los json y obtener un solo dataframe

In [20]:
def process_json_files(json_directory):
    # Lista de archivos JSON en el directorio especificado
    json_files = glob.glob(json_directory + '/*.json')
    
    # Inicializar un DataFrame vacío para almacenar los resultados
    final_df = pd.DataFrame(columns=['name', 'address', 'gmap_id', 'category', 'avg_rating'])

    # Iterar sobre los archivos JSON
    for json_file in json_files:
        # Leer el JSON y seleccionar las columnas deseadas
        df = pd.read_json(json_file, lines=True)
        df = df[['name', 'address', 'gmap_id', 'category', 'avg_rating']]
        
        # Eliminar filas con valores nulos en 'address' y 'category'
        df = df.dropna(subset=['address'])
        df = df.dropna(subset=['category'])
        
        # La columna state la reemplazo por address y normalizo por el ISO del estado
        df['state'] = df['address'].str.upper()
        df['state'] = df['state'].str.replace('FLORIDA', 'FL')
        
        # Revisar los registros que contienen el ISO o el nombre de los estados y reemplazar por el ISO
        df['state'] = df.apply(lambda row: 'FL' if row['state'] is not None and ('FL' in row['state'] or 'FLORIDA' in row['state']) else row['state'], axis=1)

        # Filtro por etado de Florida
        df = df[df['state'].isin([ 'FL'])]

        
        # Verificar si alguna palabra clave está presente en las listas de strings de la columna 'category'
        df['contains_keyword'] = df['category'].apply(buscar_coincidencias)  
        
        # Filtrar el DataFrame para obtener solo las filas que contienen palabras clave (los que sean TRUE)
        df = df[df['contains_keyword']]

        # Borrar duplicados de gmap_id:
        df.drop_duplicates(subset="gmap_id", inplace=True)
        
        # Concatenar los resultados en el DataFrame final
        final_df = pd.concat([final_df, df])

    return final_df

In [21]:
# Llamar a la función con el directorio de JSON
json_directory = 'Data/Metadata'  # Ajusta la ruta a tu directorio de JSON
df_metadata=process_json_files(json_directory)

In [23]:
df_metadata.to_csv('gmap_metadata_florida.csv', index=False)

## Review Estados

### Para el estado de Florida

A partir de los json de los estados seleccionamos los 19 json que hacen referencia a la información del estado de Florida

In [46]:
#Leemos los archivos
df1=pd.read_json('Data/Estados/Florida/1.json', lines=True)
df2=pd.read_json('Data/Estados/Florida/2.json', lines=True)
df3=pd.read_json('Data/Estados/Florida/3.json', lines=True)
df4=pd.read_json('Data/Estados/Florida/4.json', lines=True)
df5=pd.read_json('Data/Estados/Florida/5.json', lines=True)
df6=pd.read_json('Data/Estados/Florida/6.json', lines=True)
df7=pd.read_json('Data/Estados/Florida/7.json', lines=True)
df8=pd.read_json('Data/Estados/Florida/8.json', lines=True)
df9=pd.read_json('Data/Estados/Florida/9.json', lines=True)
df10=pd.read_json('Data/Estados/Florida/10.json', lines=True)
df11=pd.read_json('Data/Estados/Florida/11.json', lines=True)
df12=pd.read_json('Data/Estados/Florida/12.json', lines=True)
df13=pd.read_json('Data/Estados/Florida/13.json', lines=True)
df14=pd.read_json('Data/Estados/Florida/14.json', lines=True)
df15=pd.read_json('Data/Estados/Florida/15.json', lines=True)
df16=pd.read_json('Data/Estados/Florida/16.json', lines=True)
df17=pd.read_json('Data/Estados/Florida/17.json', lines=True)
df18=pd.read_json('Data/Estados/Florida/18.json', lines=True)
df19=pd.read_json('Data/Estados/Florida/19.json', lines=True)

In [47]:
#Concatenamos los archivos en un único dataset
df_florida=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18, df19], axis=0,join='inner')

In [48]:
#Verificamos la correcta concatenación
df_florida

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.014719e+20,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,None,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,None,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,1467907819586,1,I was told he is a good doctor. I was trying t...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,James Haynes,1480683415081,5,Takes the time to actually get to know his pat...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
...,...,...,...,...,...,...,...,...
149995,1.039909e+20,J. B. Oliveira,1503293272680,5,It is a buyers paradise. Hundred places inside...,None,None,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149996,1.124797e+20,Adam K,1514763473117,5,Great place to go especially if you're in the ...,None,None,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149997,1.103771e+20,English People,1531250213447,5,I love this place so much and come here alot a...,None,None,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149998,1.099194e+20,Shiva K,1532621720862,4,"Plenty of shops with excellent variety, ample ...",None,None,0x88d908feeaeded1f:0x29c1bf6101ce3a42


In [49]:
#Observamos las dimensiones del dataframe
df_florida.shape

(2850000, 8)

In [50]:
#Identificamos los tipos de datos del dataframe
df_florida.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2850000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 195.7+ MB


In [51]:
#Revisamos los nulos
df_florida.isnull().sum()

user_id          0
name             0
time             0
rating           0
text       1079510
pics       2745810
resp       2394601
gmap_id          0
dtype: int64

In [52]:
#Selecciono solo las columnas que quiero utilizar para mi análisis
df_florida = df_florida[['user_id','time','name','rating','text','gmap_id']]

In [53]:
#Verifico la correcta selección de las columnas
df_florida.head()

,user_id,time,name,rating,text,gmap_id
0,1.014719e+20,1628003250740,Julie A. Gerber,1,Update: Their “reply” to my review amounted to...,0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,1595031217005,Martin Sheffield,5,He's a knowledgeable doctor but the way he run...,0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,1522924253567,Brian Truett,5,"Best doctor I've ever had, I never wait to be ...",0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,1467907819586,Tina Sun,1,I was told he is a good doctor. I was trying t...,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,1480683415081,James Haynes,5,Takes the time to actually get to know his pat...,0x8893863ea87bd5dd:0x9383ebf973e74abb
...,...,...,...,...,...,...
149995,1.039909e+20,1503293272680,J. B. Oliveira,5,It is a buyers paradise. Hundred places inside...,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149996,1.124797e+20,1514763473117,Adam K,5,Great place to go especially if you're in the ...,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149997,1.103771e+20,1531250213447,English People,5,I love this place so much and come here alot a...,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149998,1.099194e+20,1532621720862,Shiva K,4,"Plenty of shops with excellent variety, ample ...",0x88d908feeaeded1f:0x29c1bf6101ce3a42


In [54]:
#Pasamos el tipo de dato de la columna time de int a datetime
df_florida['time'] = pd.to_datetime(df_florida['time'], unit='ms')

C:\Users\HP\AppData\Local\Temp\ipykernel_9144\3097459157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_florida['time'] = pd.to_datetime(df_florida['time'], unit='ms')


In [55]:
#Selecciono unicamente la fecha (sin la hora y minutos)
df_florida['time'] = df_florida['time'].dt.date

C:\Users\HP\AppData\Local\Temp\ipykernel_9144\2586066530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_florida['time'] = df_florida['time'].dt.date


In [56]:
#Observamos que se haya modificado la columna time correctamente
df_florida.head()

,user_id,time,name,rating,text,gmap_id
0,1.014719e+20,2021-08-03,Julie A. Gerber,1,Update: Their “reply” to my review amounted to...,0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,2020-07-18,Martin Sheffield,5,He's a knowledgeable doctor but the way he run...,0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,2018-04-05,Brian Truett,5,"Best doctor I've ever had, I never wait to be ...",0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,2016-07-07,Tina Sun,1,I was told he is a good doctor. I was trying t...,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,2016-12-02,James Haynes,5,Takes the time to actually get to know his pat...,0x8893863ea87bd5dd:0x9383ebf973e74abb
...,...,...,...,...,...,...
149995,1.039909e+20,2017-08-21,J. B. Oliveira,5,It is a buyers paradise. Hundred places inside...,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149996,1.124797e+20,2017-12-31,Adam K,5,Great place to go especially if you're in the ...,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149997,1.103771e+20,2018-07-10,English People,5,I love this place so much and come here alot a...,0x88d908feeaeded1f:0x29c1bf6101ce3a42
149998,1.099194e+20,2018-07-26,Shiva K,4,"Plenty of shops with excellent variety, ample ...",0x88d908feeaeded1f:0x29c1bf6101ce3a42


In [57]:
#Hacemos un merge entre los estados de florida y la metadata
df_state_florida=df_metadata.merge(df_florida, on='gmap_id', how='inner')

In [58]:
#Observamos el merge
df_state_florida

,name_x,address,gmap_id,category,avg_rating,state,contains_keyword,user_id,time,name_y,rating,text
0,Gormley's on the river | Modern cuisine in his...,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",4.3,FL,True,1.130683e+20,2018-09-17,Edwin Ashurst,5,The salads were lackluster though with well wr...
1,Gormley's on the river | Modern cuisine in his...,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",4.3,FL,True,1.059416e+20,2018-09-20,Douglas Gummere,1,"Love this place. Great little restaurant, with..."
2,Gormley's on the river | Modern cuisine in his...,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",4.3,FL,True,1.098577e+20,2018-08-22,David Powell,5,A classy restaurant to match the charm of the ...
3,Gormley's on the river | Modern cuisine in his...,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",4.3,FL,True,1.147377e+20,2019-04-17,Hillary Hutchins,5,Great atmosphere and good food
4,Gormley's on the river | Modern cuisine in his...,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",4.3,FL,True,1.089870e+20,2019-05-20,Ajay patel,5,Good food
...,...,...,...,...,...,...,...,...,...,...,...,...
1018611,Boston Market,"Boston Market, 250 E Michigan St, Orlando, FL ...",0x88e77b72f6649745:0x79b4f1ce48d76510,"[American restaurant, Caterer, Chicken restaur...",4.0,FL,True,1.005875e+20,2019-05-19,Christy Flanagan,4,None
1018612,Boston Market,"Boston Market, 250 E Michigan St, Orlando, FL ...",0x88e77b72f6649745:0x79b4f1ce48d76510,"[American restaurant, Caterer, Chicken restaur...",4.0,FL,True,1.102315e+20,2019-08-25,Mspoetry,3,None
1018613,Boston Market,"Boston Market, 250 E Michigan St, Orlando, FL ...",0x88e77b72f6649745:0x79b4f1ce48d76510,"[American restaurant, Caterer, Chicken restaur...",4.0,FL,True,1.056372e+20,2019-12-02,None Done,3,None
1018614,Boston Market,"Boston Market, 250 E Michigan St, Orlando, FL ...",0x88e77b72f6649745:0x79b4f1ce48d76510,"[American restaurant, Caterer, Chicken restaur...",4.0,FL,True,1.001089e+20,2017-11-27,David Santillo,3,None


In [59]:
#Eliminamos las columnas que no utilizaremos
df_state_florida=df_state_florida.drop(columns=['address','avg_rating', 'contains_keyword', 'name_y'])

In [60]:
#Revisamos si hay valores nulos
df_state_florida.isnull().sum()

name_x           0
gmap_id          0
category         0
state            0
user_id          0
time             0
rating           0
text        389356
dtype: int64

En el dataframe de reviews me interesa tener información de las reseñas, por lo tanto eliminaré los valores nulos en la columna text

In [61]:
#Eliminamos los nulos de la columna text
df_state_florida=df_state_florida.dropna(subset=['text'])

In [62]:
#Verificamos que se hayan eliminado los nulos correctamente
df_state_florida.isnull().sum()

name_x      0
gmap_id     0
category    0
state       0
user_id     0
time        0
rating      0
text        0
dtype: int64

In [63]:
#Revisamos las nuevas dimensiones del dataframe
df_state_florida.shape

(629260, 8)

Para nuestro análisis nos centraremos en un período de tiempo desde comienzos del 2017 y hasta la fecha de la review más actual

In [65]:
#Identificamos la fecha máxima y mínima para filtrar
date_min = pd.to_datetime('2017-01-01').date()
date_max = pd.to_datetime('2021-09-08').date()

In [66]:
#Filtramos el dataset para la fecha seleccionada
df_state_florida_date = df_state_florida[(df_state_florida['time']>= date_min) & (df_state_florida['time']<= date_max)]

In [67]:
#Verificamos el filtro
df_state_florida_date

,name_x,gmap_id,category,state,user_id,time,rating,text
0,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",FL,1.130683e+20,2018-09-17,5,The salads were lackluster though with well wr...
1,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",FL,1.059416e+20,2018-09-20,1,"Love this place. Great little restaurant, with..."
2,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",FL,1.098577e+20,2018-08-22,5,A classy restaurant to match the charm of the ...
3,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",FL,1.147377e+20,2019-04-17,5,Great atmosphere and good food
4,Gormley's on the river | Modern cuisine in his...,0x8894b5a7a7909725:0xe4687299fd288188,"[Modern French restaurant, Restaurant]",FL,1.089870e+20,2019-05-20,5,Good food
...,...,...,...,...,...,...,...,...
1018351,Boston Market,0x88e77b72f6649745:0x79b4f1ce48d76510,"[American restaurant, Caterer, Chicken restaur...",FL,1.178976e+20,2019-06-19,5,(Translated by Google) I loved..\n\n(Original)...
1018352,Boston Market,0x88e77b72f6649745:0x79b4f1ce48d76510,"[American restaurant, Caterer, Chicken restaur...",FL,1.157899e+20,2019-07-04,2,(Translated by Google) They are very expensive...
1018353,Boston Market,0x88e77b72f6649745:0x79b4f1ce48d76510,"[American restaurant, Caterer, Chicken restaur...",FL,1.086188e+20,2019-03-31,5,(Translated by Google) Excellent shop.\n\n(Ori...
1018354,Boston Market,0x88e77b72f6649745:0x79b4f1ce48d76510,"[American restaurant, Caterer, Chicken restaur...",FL,1.147855e+20,2019-06-21,5,(Translated by Google) luvvvv\n\n(Original)\nl...


In [ ]:
#Revisamos las nuevas dimensiones del dataframe
df_state_florida_date.shape

In [68]:
#Exportamos el archivo a csv
df_state_florida_date.to_csv('gmap_state_florida.csv', index=False)